# Goals:

*   Understand how the DAVIS images are being resized and use cropping instead.
*   When displaying SSIM and PSNR scores, do not display the images/frames

# "Play" the Video on the Colab Notebook. This Video Will Be Our Ground Truth

In [1]:
import os
from moviepy.editor import ImageSequenceClip
from PIL import Image

folder_path = 'DAVIS_image_files_full_resolution'
image_files = os.listdir(folder_path)
image_files.sort()

# Create a list to store image frames
cleanFrames4k = []
for image_file_4k in image_files:
    image_path_4k = os.path.join(folder_path, image_file_4k)
    cleanFrames4k.append(image_path_4k)

# Create a list to store the image frames in size 480p
cleanFrames480 = []
target_resolution = (0, 0, 854, 480)
for image_path_480 in cleanFrames4k:
    image_480 = Image.open(image_path_480)
    resized_image_480 = image_480.crop(target_resolution)
    cleanFrames480.append(resized_image_480)

# Create a list to store the image frames in size 720p
cleanFrames720 = []
target_resolution = (0, 0, 1280, 720)
for image_path_720 in cleanFrames4k:
    image_720 = Image.open(image_path_720)
    resized_image_720 = image_720.crop(target_resolution)
    cleanFrames720.append(resized_image_720)

# Add Salt and Pepper Noise to the Video

In [2]:
import os
import cv2
import numpy as np
from moviepy.editor import ImageSequenceClip

In [3]:
def add_salt_pepper_noise(image, noise_level):
    noisy_image = np.copy(image)
    height, width, _ = noisy_image.shape
    num_pixels = int(noise_level * height * width)

    # Generate random coordinates
    random_coordinates = np.random.randint(0, high=(width, height), size=(num_pixels, 2))

    # Generate random values for salt (white pixels) and pepper (black pixels)
    salt_value = [255, 255, 255]
    pepper_value = [0, 0, 0]

    # Add salt and pepper noise to the random coordinates
    for x, y in random_coordinates:
        noisy_image[y, x] = salt_value if np.random.rand() < 0.5 else pepper_value

    return noisy_image

In [4]:
# Create a list to store image frames
noisyFrames4k = []
for image_file_4k in image_files:
    image_path_4k = os.path.join(folder_path, image_file_4k)

    # Read the image
    image_4k = cv2.imread(image_path_4k)

    # Convert color channel ordering from BGR to RGB
    image_4k = cv2.cvtColor(image_4k, cv2.COLOR_BGR2RGB)

    # Add salt and pepper noise to the image
    noisy_image_4k = add_salt_pepper_noise(image_4k, noise_level=0.10)  # Adjust the noise level as desired

    # Append the noisy image to the frames list
    noisyFrames4k.append(noisy_image_4k)

# Create a list to store image frames in 480p
noisyFrames480 = []
target_resolution = (0, 0, 854, 480)
for image_480 in noisyFrames4k:
    image_pil_480 = Image.fromarray(image_480)
    resized_image_480 = image_pil_480.crop(target_resolution)
    noisyFrames480.append(resized_image_480)

# Create a list to store image frames in 720p
noisyFrames720 = []
target_resolution = (0, 0, 1280, 720)
for image_720 in noisyFrames4k:
    image_pil_720 = Image.fromarray(image_720)
    resized_image_720 = image_pil_720.crop(target_resolution)
    noisyFrames720.append(resized_image_720)

# Measure the Noise Between the Noisy and Ground Truth Images Using SSIM and PSNR

*  SSIM > 0.9: Excellent similarity. The reconstructed/filtered image closely resembles the original image.
*  0.7 < SSIM < 0.9: Good similarity. The reconstructed/filtered image has some minor differences but still maintains a high level of similarity to the original image.
*  0.5 < SSIM < 0.7: Moderate similarity. The reconstructed/filtered image has noticeable differences but still retains some level of similarity to the original image.
*  SSIM < 0.5: Poor similarity. The reconstructed/filtered image has significant differences and may not be visually similar to the original image. differences.

*   ############################################################

*   PSNR > 40 dB: Excellent quality. The reconstructed/filtered image closely resembles the original image.
*   30 dB < PSNR < 40 dB: Good quality. The reconstructed/filtered image has some minor imperfections but is still visually similar to the original image.
*    20 dB < PSNR < 30 dB: Moderate quality. The reconstructed/filtered image has noticeable distortions or noise but is still recognizable.
*    PSNR < 20 dB: Poor quality. The reconstructed/filtered image has significant distortions or noise and may differ significantly from the original image.





In [5]:
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# Create an empty list to store the SSIM scores
ssim_scores_4k = []
psnr_scores_4k = []

for ground_image_path_4k, noisy_image_path_4k in zip(cleanFrames4k, noisyFrames4k):

    ground_image_4k = Image.open(ground_image_path_4k)
    noisy_image_4k = Image.fromarray(noisy_image_path_4k)

    # PSNR -- Convert to NumPy arrays
    ground_image_4k_np = np.array(ground_image_4k)
    noisy_image_4k_np = np.array(noisy_image_4k)

    # SSIM -- Convert to gray-scale and NumPy arrays
    ground_image_4k_gray = np.array(ground_image_4k.convert('L'))
    noisy_image_4k_gray = np.array(noisy_image_4k.convert('L'))

    # Calculate SSIM
    ssim_score = ssim(ground_image_4k_gray, noisy_image_4k_gray)

    # Calculate PSNR
    psnr_score = psnr(ground_image_4k_np, noisy_image_4k_np)

    # Append the SSIM score to the list
    ssim_scores_4k.append(ssim_score)

    # Append the PSNR score to the list
    psnr_scores_4k.append(psnr_score)

# Calculate the average SSIM score (4K)
average_ssim_4k = np.mean(ssim_scores_4k)
print(f"Average SSIM Score 4K: {average_ssim_4k:.4f}")

# Calculate the average PSNR score (4k)
average_psnr_4k = np.mean(psnr_scores_4k)
print(f"Average PSNR Score 4K: {average_psnr_4k:.4f}")

Average SSIM Score 4K: 0.0802
Average PSNR Score 4K: 15.5980


In [6]:
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# Create an empty list to store the SSIM scores
ssim_scores_480 = []
psnr_scores_480 = []

for ground_image_path_480, noisy_image_path_480 in zip(cleanFrames480, noisyFrames480):

    ground_image_480 = ground_image_path_480
    noisy_image_480 = noisy_image_path_480

    # PSNR -- Convert to NumPy arrays
    ground_image_480_np = np.array(ground_image_480)
    noisy_image_480_np = np.array(noisy_image_480)

    # SSIM -- Convert to gray-scale and NumPy arrays
    ground_image_480_gray = np.array(ground_image_480.convert('L'))
    noisy_image_480_gray = np.array(noisy_image_480.convert('L'))

    # Calculate SSIM
    ssim_score = ssim(ground_image_480_gray, noisy_image_480_gray)

    # Calculate PSNR
    psnr_score = psnr(ground_image_480_np, noisy_image_480_np)

    # Append the SSIM score to the list
    ssim_scores_480.append(ssim_score)

    # Append the PSNR score to the list
    psnr_scores_480.append(psnr_score)

# Calculate the average SSIM score (480)
average_ssim_480 = np.mean(ssim_scores_480)
print(f"Average SSIM Score 480p: {average_ssim_480:.4f}")

# Calculate the average PSNR score (480)
average_psnr_480 = np.mean(psnr_scores_480)
print(f"Average PSNR Score 480p: {average_psnr_480:.4f}")

Average SSIM Score 480p: 0.0611
Average PSNR Score 480p: 15.5736


In [7]:
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# Create an empty list to store the SSIM scores
ssim_scores_720 = []
psnr_scores_720 = []

for ground_image_path_720, noisy_image_path_720 in zip(cleanFrames720, noisyFrames720):

    ground_image_720 = ground_image_path_720
    noisy_image_720 = noisy_image_path_720

    # PSNR -- Convert to NumPy arrays
    ground_image_720_np = np.array(ground_image_720)
    noisy_image_720_np = np.array(noisy_image_720)

    # SSIM -- Convert to gray-scale and NumPy arrays
    ground_image_720_gray = np.array(ground_image_720.convert('L'))
    noisy_image_720_gray = np.array(noisy_image_720.convert('L'))

    # Calculate SSIM
    ssim_score = ssim(ground_image_720_gray, noisy_image_720_gray)

    # Calculate PSNR
    psnr_score = psnr(ground_image_720_np, noisy_image_720_np)

    # Append the SSIM score to the list
    ssim_scores_720.append(ssim_score)

    # Append the PSNR score to the list
    psnr_scores_720.append(psnr_score)

# Calculate the average SSIM score (720)
average_ssim_720 = np.mean(ssim_scores_720)
print(f"Average SSIM Score 720p: {average_ssim_720:.4f}")

# Calculate the average PSNR score (720)
average_psnr_720 = np.mean(psnr_scores_720)
print(f"Average PSNR Score 720p: {average_psnr_720:.4f}")

Average SSIM Score 720p: 0.0732
Average PSNR Score 720p: 15.5191


# Apply a Gaussian Filter to the Frames and Measure How Long It Takes to Process Each Frame

In [8]:
from skimage.filters import gaussian
import matplotlib.pyplot as plt
import time

# Define the parameters for the Gaussian filter --> These parameters can be edited to improve SSIM and PSNR scores.
sigma = 1.5 # Standard deviation of the Gaussian kernel
truncate = 2.0 # Truncation of the Gaussian kernel

# List to store the filtered images
filteredFrames4k = []

# List to store the execution times
executionTimes_4k = []

# Apply Gaussian filter to each noisy image
for noisy_image_path_4k in noisyFrames4k:
    noisy_image_arr_4k = noisy_image_path_4k

    # Normalize the pixel values to the range of 0 to 1
    normalized_image_arr_4k = noisy_image_arr_4k.astype(np.float32) / 255.0

    # Apply the Gaussian filter
    start_time = time.time()
    filtered_image_4k = gaussian(normalized_image_arr_4k, sigma=sigma, truncate=truncate, multichannel=True)
    end_time = time.time()

    # Scale the pixel values back to the range of 0 to 255
    filtered_image_scaled_4k = (filtered_image_4k * 255.0).clip(0, 255).astype(np.uint8)

    # Convert the filtered image array back to PIL Image
    filtered_image_pil_4k = Image.fromarray(filtered_image_scaled_4k)

    # Add the filtered image to the list
    filteredFrames4k.append(filtered_image_pil_4k)

    # Calculate the execution time
    execution_time_4k = end_time - start_time

    # Append the execution time to the list
    executionTimes_4k.append(execution_time_4k)

# Calculate the average execution time scores
average_execution_time_4k = np.mean(executionTimes_4k)
print(f"Average Execution Time (4K): {average_execution_time_4k:.4f} seconds\n")

Average Execution Time (4K): 0.4543 seconds



In [9]:
# Define the parameters for the Gaussian filter --> These parameters can be edited to improve SSIM and PSNR scores.
sigma = 1.5 # Standard deviation of the Gaussian kernel
truncate = 2.0 # Truncation of the Gaussian kernel

# List to store the filtered images
filteredFrames480 = []

# List to store the execution times
executionTimes_480 = []

# Apply Gaussian filter to each noisy image
for noisy_image_path_480 in noisyFrames480:
    noisy_image_arr_480 = np.array(noisy_image_path_480)

    # Normalize the pixel values to the range of 0 to 1
    normalized_image_arr_480 = noisy_image_arr_480.astype(np.float32) / 255.0

    # Apply the Gaussian filter
    start_time = time.time()
    filtered_image_480 = gaussian(normalized_image_arr_480, sigma=sigma, truncate=truncate, multichannel=True)
    end_time = time.time()

    # Scale the pixel values back to the range of 0 to 255
    filtered_image_scaled_480 = (filtered_image_480 * 255.0).clip(0, 255).astype(np.uint8)

    # Convert the filtered image array back to PIL Image
    filtered_image_pil_480 = Image.fromarray(filtered_image_scaled_480)

    # Add the filtered image to the list
    filteredFrames480.append(filtered_image_pil_480)

    # Calculate the execution time
    execution_time_480 = end_time - start_time

    # Append the execution time to the list
    executionTimes_480.append(execution_time_480)

# Calculate the average execution time scores
average_execution_time_480 = np.mean(executionTimes_480)
print(f"Average Execution Time (480p): {average_execution_time_480:.4f} seconds\n")

Average Execution Time (480p): 0.0118 seconds



In [10]:
# Define the parameters for the Gaussian filter --> These parameters can be edited to improve SSIM and PSNR scores.
sigma = 1.5 # Standard deviation of the Gaussian kernel
truncate = 2.0 # Truncation of the Gaussian kernel

# List to store the filtered images
filteredFrames720 = []

# List to store the execution times
executionTimes_720 = []

# Apply Gaussian filter to each noisy image
for noisy_image_path_720 in noisyFrames720:
    noisy_image_arr_720 = np.array(noisy_image_path_720)

    # Normalize the pixel values to the range of 0 to 1
    normalized_image_arr_720 = noisy_image_arr_720.astype(np.float32) / 255.0

    # Apply the Gaussian filter
    start_time = time.time()
    filtered_image_720 = gaussian(normalized_image_arr_720, sigma=sigma, truncate=truncate, multichannel=True)
    end_time = time.time()

    # Scale the pixel values back to the range of 0 to 255
    filtered_image_scaled_720 = (filtered_image_720 * 255.0).clip(0, 255).astype(np.uint8)

    # Convert the filtered image array back to PIL Image
    filtered_image_pil_720 = Image.fromarray(filtered_image_scaled_720)

    # Add the filtered image to the list
    filteredFrames720.append(filtered_image_pil_720)

    # Calculate the execution time
    execution_time_720 = end_time - start_time

    # Append the execution time to the list
    executionTimes_720.append(execution_time_720)

# Calculate the average execution time scores
average_execution_time_720 = np.mean(executionTimes_720)
print(f"Average Execution Time (720p): {average_execution_time_720:.4f} seconds\n")

Average Execution Time (720p): 0.0325 seconds



# Compare the Filtered Video Quantitatively (SSIM, PSNR) to the Ground Truth Video

*  SSIM > 0.9: Excellent similarity. The reconstructed/filtered image closely resembles the original image.
*  0.7 < SSIM < 0.9: Good similarity. The reconstructed/filtered image has some minor differences but still maintains a high level of similarity to the original image.
*  0.5 < SSIM < 0.7: Moderate similarity. The reconstructed/filtered image has noticeable differences but still retains some level of similarity to the original image.
*  SSIM < 0.5: Poor similarity. The reconstructed/filtered image has significant differences and may not be visually similar to the original image. differences.

*   ############################################################

*   PSNR > 40 dB: Excellent quality. The reconstructed/filtered image closely resembles the original image.
*   30 dB < PSNR < 40 dB: Good quality. The reconstructed/filtered image has some minor imperfections but is still visually similar to the original image.
*    20 dB < PSNR < 30 dB: Moderate quality. The reconstructed/filtered image has noticeable distortions or noise but is still recognizable.
*    PSNR < 20 dB: Poor quality. The reconstructed/filtered image has significant distortions or noise and may differ significantly from the original image.


In [11]:
# Create an empty list to store the SSIM scores
ssim_scores_4k = []
psnr_scores_4k = []

for ground_image_path_4k, filtered_image_path_4k in zip(cleanFrames4k, filteredFrames4k):

    ground_image_4k = Image.open(ground_image_path_4k)
    filtered_image_4k = filtered_image_path_4k

    # PSNR -- Convert to NumPy arrays
    ground_image_4k_np = np.array(ground_image_4k)
    filtered_image_4k_np = np.array(filtered_image_4k)

    # SSIM -- Convert to gray-scale and NumPy arrays
    ground_image_4k_gray = np.array(ground_image_4k.convert('L'))
    filtered_image_4k_gray = np.array(filtered_image_4k.convert('L'))

    # Calculate SSIM
    ssim_score = ssim(ground_image_4k_gray, filtered_image_4k_gray)

    # Calculate PSNR
    psnr_score = psnr(ground_image_4k_np, filtered_image_4k_np)

    # Append the SSIM score to the list
    ssim_scores_4k.append(ssim_score)

    # Append the PSNR score to the list
    psnr_scores_4k.append(psnr_score)

# Calculate the average SSIM score (4K)
average_ssim_4k = np.mean(ssim_scores_4k)
print(f"Average SSIM Score 4K: {average_ssim_4k:.4f}")

# Calculate the average PSNR score (4k)
average_psnr_4k = np.mean(psnr_scores_4k)
print(f"Average PSNR Score 4K: {average_psnr_4k:.4f}")

Average SSIM Score 4K: 0.6317
Average PSNR Score 4K: 28.4885


In [12]:
# Create an empty list to store the SSIM scores
ssim_scores_480 = []
psnr_scores_480 = []

for ground_image_path_480, filtered_image_path_480 in zip(cleanFrames480, filteredFrames480):

    ground_image_480 = ground_image_path_480
    filtered_image_480 = filtered_image_path_480

    # PSNR -- Convert to NumPy arrays
    ground_image_480_np = np.array(ground_image_480)
    filtered_image_480_np = np.array(filtered_image_480)

    # SSIM -- Convert to gray-scale and NumPy arrays
    ground_image_480_gray = np.array(ground_image_480.convert('L'))
    filtered_image_480_gray = np.array(filtered_image_480.convert('L'))

    # Calculate SSIM
    ssim_score = ssim(ground_image_480_gray, filtered_image_480_gray)

    # Calculate PSNR
    psnr_score = psnr(ground_image_480_np, filtered_image_480_np)

    # Append the SSIM score to the list
    ssim_scores_480.append(ssim_score)

    # Append the PSNR score to the list
    psnr_scores_480.append(psnr_score)

# Calculate the average SSIM score (480)
average_ssim_480 = np.mean(ssim_scores_480)
print(f"Average SSIM Score 480p: {average_ssim_480:.4f}")

# Calculate the average PSNR score (480)
average_psnr_480 = np.mean(psnr_scores_480)
print(f"Average PSNR Score 480p: {average_psnr_480:.4f}")

Average SSIM Score 480p: 0.6054
Average PSNR Score 480p: 28.6513


In [13]:
# Create an empty list to store the SSIM scores
ssim_scores_720 = []
psnr_scores_720 = []

for ground_image_path_720, filtered_image_path_720 in zip(cleanFrames720, filteredFrames720):

    ground_image_720 = ground_image_path_720
    filtered_image_720 = filtered_image_path_720

    # PSNR -- Convert to NumPy arrays
    ground_image_720_np = np.array(ground_image_720)
    filtered_image_720_np = np.array(filtered_image_720)

    # SSIM -- Convert to gray-scale and NumPy arrays
    ground_image_720_gray = np.array(ground_image_720.convert('L'))
    filtered_image_720_gray = np.array(filtered_image_720.convert('L'))

    # Calculate SSIM
    ssim_score = ssim(ground_image_720_gray, filtered_image_720_gray)

    # Calculate PSNR
    psnr_score = psnr(ground_image_720_np, filtered_image_720_np)

    # Append the SSIM score to the list
    ssim_scores_720.append(ssim_score)

    # Append the PSNR score to the list
    psnr_scores_720.append(psnr_score)

# Calculate the average SSIM score (720)
average_ssim_720 = np.mean(ssim_scores_720)
print(f"Average SSIM Score 720p: {average_ssim_720:.4f}")

# Calculate the average PSNR score (720)
average_psnr_720 = np.mean(psnr_scores_720)
print(f"Average PSNR Score 720p: {average_psnr_720:.4f}")

Average SSIM Score 720p: 0.6155
Average PSNR Score 720p: 28.4252
